In [ ]:
import os
import numpy as np
import pandas as pd
import time

from regain.covariance import GraphicalLasso as rg_GL
from sklearn.covariance import GraphicalLasso as sk_GL


from regain_benchmark import regain_time
from sklearn_benchmark import sklearn_time
from gglasso_benchmark import gglasso_time

from utilita import network_generation, model_solution, benchmark_parameters 
from utilita import sparsity_benchmark, dict_shape, calc_hamming_dict, benchmarks_dataframe

from plots import plot_accuracy, plot_scalability, plot_lambdas, plot_bm

from tqdm.contrib import tzip
from tqdm.contrib.itertools import product

pd.set_option('display.max_columns', None)

The notebook illustrates the comparison of methods solving [Graphical lasso](https://en.wikipedia.org/wiki/Graphical_lasso) problem implemented in [regain](https://github.com/fdtomasi/regain), [gglasso](https://github.com/fabian-sp/GGLasso/tree/o-benchmarking) and [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.covariance.graphical_lasso.html) libraries, respectivelly.

### Power networks

We generate a precision matrix following the approach of [Danaher et al.,
2014](https://arxiv.org/abs/1111.0324) where blockwise [subnetworks](https://en.wikipedia.org/wiki/Scale-free_network) are sampled from a power-law degree distribution.

In [ ]:
S_dict=dict()
X_dict=dict()
Theta_dict=dict()

#p_list=[100, 500, 1000, 2000]
#N_list=[110, 550, 1100, 2200]

p_list=[100, 200]
N_list=[110, 220]

print(" Power network generation ".center(40, '-'))

for p, N in tzip(p_list, N_list):
    start = time.perf_counter()
    S, X, Theta = network_generation(p, N, M=10)
    end = time.perf_counter()
    print("p: %5d, N : %5d, Time : %5.4f" % (p, N, end-start))

    S_dict[p, N] = S.copy()
    X_dict[p, N] = X.copy()
    Theta_dict[p, N] = Theta.copy()

In [ ]:
print("\n Shape of S_i:", dict_shape(S_dict))
print("\n Shape of X_i:", dict_shape(X_dict))
print("\n Shape of Theta_i:", dict_shape(Theta_dict))

### Hyperparameters

Since the solution of graphical lasso varies in the presented methods, we pick hyperparameters which allow us to have approximatelly similar solutions for all the solvers, so one can compare the methods.


We take the following steps to achieve comparable soltuions of graphical lasso:
- Solve graphical problem by one of the methods at the high level of precision ( $Z^{*}$)
- Solve graphical problem by each of the solvers at the lower level of precision ($Z_{i}$)
- Calculate the normalized Eucledean distance ($\text{accuracy}$) between $Z^{*}$ and solution $Z_{i}$:
$$
\begin{align}
\text{accuracy} = \frac{\lVert Z^{*} - Z_{i} \rVert}{\lVert Z^{*} \rVert}
\end{align}
$$

In [ ]:
lambda_list = [0.5, 0.1, 0.05]
sk_params, rg_params, gglasso_params, lambda_list = benchmark_parameters(lambda_list = lambda_list)

### Model solution

In [ ]:
model_time_dict = dict()
model_Z_dict = dict()
reference_solver = "regain"

print(f"Solving for a reference solution with solver {reference_solver}:")

for X, l1 in product(list(X_dict.values()), lambda_list):
    
    Z, Z_time, info = model_solution(solver=reference_solver, X=X, lambda1=l1)
    
    key = "p_" + str(X.shape[1]) + "_N_" + str(X.shape[0]) + "_l1_" + str(l1)
    model_time_dict.update({key: Z_time})
    model_Z_dict.update({key: Z})

In [ ]:
time_dict = dict()
accuracy_dict = dict()
Z_dict = dict()
iter_dict = dict()

In [ ]:
# how often to repeat each call (averaging runtime)
n_iter = 2

### GGLasso

In [ ]:
for X, S in tzip(list(X_dict.values()), list(S_dict.values())):
    Omega_0 = np.eye(len(S))
    gg_result = gglasso_time(S=S, X=X, Omega_0=Omega_0, Z=model_Z_dict, lambda_list=lambda_list,
                                              n_iter=n_iter, gglasso_params=gglasso_params, warm_start=False)
    
    time_dict.update(gg_result['time'])
    accuracy_dict.update(gg_result['accuracy'])
    Z_dict.update(gg_result['sol'])
    iter_dict.update(gg_result['iter'])

### Sklearn

In [ ]:
for X, S in tzip(list(X_dict.values()), list(S_dict.values())):
    sk_result = sklearn_time(X=X, Z=model_Z_dict, sk_params=sk_params, lambda_list=lambda_list, \
                                              n_iter=n_iter, max_iter=30)
    
    time_dict.update(sk_result['time'])
    accuracy_dict.update(sk_result['accuracy'])
    Z_dict.update(sk_result['sol'])
    iter_dict.update(sk_result['iter'])

### Regain

In [ ]:
for X, S in tzip(list(X_dict.values()), list(S_dict.values())):
    rg_result = regain_time(X=X, Z=model_Z_dict, rg_params=rg_params, lambda_list=lambda_list, \
                                             n_iter=n_iter, warm_start=False)
    
    time_dict.update(rg_result['time'])
    accuracy_dict.update(rg_result['accuracy'])
    Z_dict.update(rg_result['sol'])
    iter_dict.update(rg_result['iter'])

### Hamming distances

We calculate [Hamming](https://en.wikipedia.org/wiki/Hamming_distance) distance between model solution $Z^{*}$ and given solution $Z_{i}$ with a specified rounding threshold for non zero entries.

In [ ]:
hamming_dict = calc_hamming_dict(Theta_dict=Theta_dict, Z_dict=Z_dict, t_rounding=1e-8)

### Visualization

In [ ]:
df = benchmarks_dataframe(time_dict, accuracy_dict, hamming_dict, iter_dict)
df.head()

#df.to_csv('../data/synthetic/bm2000.csv')

In [ ]:
plot_bm(df, min_acc = 1e-3, lambda_list=lambda_list)

In [ ]:
fig = plot_lambdas(df, upper_bound=0.1, lower_bound=0.000001)
fig.show()

It is clear that solving graphical lasso on individual blocks is less time consuming then solving it on the entire graph.

So, one can advise to use a block version of ADMM implemented in gglasso in a high-dimensional setup.

In [ ]:
fig = plot_accuracy(df, upper_bound=0.1, lower_bound=0.0000001, lambda_filter=0.05, sortby=['p', 'time'])
fig.show()

In [ ]:
frames = sparsity_benchmark(df, upper_bound=0.01, lower_bound=0.0001, lambda_filter=0.1)
frames